In [16]:
import pandas as pd
import os
import glob
import numpy as np
import strsimpy
print('setup done!')
import hashlib
from tqdm import tqdm

setup done!


In [17]:
pd.set_option('display.max_colwidth', None)

In [19]:
levenshtein  = strsimpy.NormalizedLevenshtein()

def levenshtein_score(x,y):
    if (x=='unknown' or y =='unknown'):
        return 1
    return 1 - levenshtein.distance(x,y)

def difference_score(x,y):
    if (x=='unknown' or y =='unknown'):
        return 1
    return 1 if x==y else 0

def clust_filter(arr):
    flag = [False]*len(arr)
    cluster = []
    for i in tqdm(range(0,len(arr)-1)):
        if flag[i]==False:
            list_id = []
            list_id.append(arr[i][10])
            for j in range(i + 1, len(arr)):
                if flag[j]==False:
                    name = difference_score(arr[i][0],arr[j][0])
                    ram = difference_score(arr[i][2],arr[j][2])
                    rom = difference_score(arr[i][3],arr[j][3])
                    color = levenshtein_score(arr[i][6],arr[j][6])
                    status = difference_score(arr[i][7], arr[j][7])
                    score = 0.5 * name + 0.05 * ram + 0.15 * rom + 0.2 * color + 0.1 * status
                    if score >= 0.9:
                        flag[j] = True
                        list_id.append(arr[j][10])
            list_id = list(set(list_id))
            arr[i].append(list_id)
            cluster.append(arr[i])
            
    return cluster

In [20]:
def encrypt_string(hash_string):
    sha_signature = hashlib.sha256(hash_string.encode()).digest()
    return "".join(["{:x}".format(b) for b in sha_signature])

In [21]:
path = os.getcwd()
path = os.path.join(path, 'data/*.csv')
csv_files = glob.glob(path)

df = pd.read_csv(csv_files[0])
for i, file in enumerate(csv_files):
    if i == 0:
        continue
    tmp = pd.read_csv(file)
    df = pd.concat([df, tmp], axis=0)

In [22]:
print(f'Total records in data: {len(df)}')

Total records in data: 2531


In [23]:
df = df.drop(['Unnamed: 0'], axis=1)

In [24]:
df.drop_duplicates(inplace=True)
len(df)

2222

In [25]:
df['id'] = df['name'].astype(str) + df['url'].astype(str)  + df['rom'].astype(str) + df['color'] + \
 df['status'].astype(str) + df['store'].astype(str)
df['id'] = df['id'].apply(encrypt_string)

In [27]:
df.reset_index(drop=True, inplace=True)
df.to_json('./db/products.json')

In [28]:
arr = df.values.tolist()

In [29]:
arr[0]

['IPHONE 14 PRO MAX',
 'https://www.xtmobile.vn/iphone-14-pro-max-chinh-hang',
 6,
 128,
 31790000.0,
 '2022-12-26 20:11:52.196381',
 'YELLOW',
 1,
 'unknown',
 'xtmobile',
 'd88fc8d85a51de83abd6a8420c6ca79982df0f6d268abed743ca5d0d5b23e9a']

In [30]:
cluster = clust_filter(arr)

100%|██████████| 2221/2221 [00:02<00:00, 898.90it/s] 


In [31]:
len(cluster)

189

In [32]:
cluster[0]

['IPHONE 14 PRO MAX',
 'https://www.xtmobile.vn/iphone-14-pro-max-chinh-hang',
 6,
 128,
 31790000.0,
 '2022-12-26 20:11:52.196381',
 'YELLOW',
 1,
 'unknown',
 'xtmobile',
 'd88fc8d85a51de83abd6a8420c6ca79982df0f6d268abed743ca5d0d5b23e9a',
 ['4fb12c7d514ce576c38f8a848819ae50fd22f0c53dda88571fc1239c548a',
  '4fd21b74f022929833ac1a2ab2633ce43d4b2dad0a7bd49138bf4c498fc5c24',
  '5b41f665b4d2c587ce10fedc8d281474c9c9240651b93f0feb0fed3d8e949',
  '4f652892b31c421174d7cc66ec88d065bf3b4528fb3bfbce628d2a5edafdcfa',
  '4e8253e27fd60475e67ad462aba22140659171aea360742c551f5769fc72a',
  '8ff4eadc6ab82531196bfc56c527b568b78eb610203cf1117a7dad2ab6932a42',
  'cb9e8cec68248a511ef4e6a7b6a97a3458152e8189e394f2f11388c9ffaf64',
  'e88ed9ed89c2ac3b191dd484634c2276711f116beacedd58bb36572113e15f5f',
  'd88fc8d85a51de83abd6a8420c6ca79982df0f6d268abed743ca5d0d5b23e9a',
  '17c94788a9e95b3e516e1e3b2a29183f84221194408bfac81caabce232674ff',
  'b18cc10f384b5492a63fc0114cc08ff63b1dac9d3b0679a9b7dc54191eaa',
  '4a0773

In [25]:
df[(df['name']=='IPHONE 14 PRO MAX') & (df['rom'] == 128) & (df['color']=='YELLOW')]

,name,url,ram,rom,price,createdDate,color,status,url_img,store,id
0,IPHONE 14 PRO MAX,https://www.xtmobile.vn/iphone-14-pro-max-chinh-hang,6,128,31790000.0,2022-12-26 20:11:52.196381,YELLOW,1,unknown,xtmobile,d88fc8d85a51de83abd6a8420c6ca79982df0f6d268abed743ca5d0d5b23e9a
212,IPHONE 14 PRO MAX,https://www.xtmobile.vn/iphone-14-pro-max-128gb-likenew,6,128,28790000.0,2022-12-26 20:11:52.196381,YELLOW,1,unknown,xtmobile,17c94788a9e95b3e516e1e3b2a29183f84221194408bfac81caabce232674ff
996,IPHONE 14 PRO MAX,https://onewaymobile.vn/iphone-14-series/dien-thoai-apple-iphone-14-promax-128gb-new100-vna-dp824.html,6,128,31990000.0,2022-12-26 19:48:41.443815,YELLOW,0,unknown,onewaymobile,de842d852fdaf2a87a71701361e6b8c7a9b35922913458a7779f9e8d7761
1223,IPHONE 14 PRO MAX,https://24hstore.vn/iphone-14-pro-max-moi/iphone-14-pro-max-1tb-p6626,6,128,31790000.0,2022-12-26 19:39:48.431188,YELLOW,0,unknown,24hstore,e88ed9ed89c2ac3b191dd484634c2276711f116beacedd58bb36572113e15f5f
1295,IPHONE 14 PRO MAX,https://24hstore.vn/iphone-14-pro-max-moi/iphone-14-pro-max-512gb-p6628,6,128,31790000.0,2022-12-26 19:39:48.431188,YELLOW,0,unknown,24hstore,8ff4eadc6ab82531196bfc56c527b568b78eb610203cf1117a7dad2ab6932a42
1355,IPHONE 14 PRO MAX,https://24hstore.vn/iphone-14-pro-max-cu/iphone-14-pro-max-256gb-cu-p7217,6,128,27090000.0,2022-12-26 19:39:48.431188,YELLOW,0,unknown,24hstore,4e8253e27fd60475e67ad462aba22140659171aea360742c551f5769fc72a
1436,IPHONE 14 PRO MAX,https://24hstore.vn/iphone-14-pro-max-cu/iphone-14-pro-max-128gb-cu-p6951,6,128,27090000.0,2022-12-26 19:39:48.431188,YELLOW,0,unknown,24hstore,4f652892b31c421174d7cc66ec88d065bf3b4528fb3bfbce628d2a5edafdcfa
1450,IPHONE 14 PRO MAX,https://24hstore.vn/iphone-14-pro-max-moi/iphone-14-pro-max-128gb-p6630,6,128,31790000.0,2022-12-26 19:39:48.431188,YELLOW,0,unknown,24hstore,4a0773cdbba842d2d5a2aab854af1f53a8bdea45f0c32baadb95652ed72df


In [33]:
df[df['id'].isin(['4fb12c7d514ce576c38f8a848819ae50fd22f0c53dda88571fc1239c548a',
  '4fd21b74f022929833ac1a2ab2633ce43d4b2dad0a7bd49138bf4c498fc5c24',
  '5b41f665b4d2c587ce10fedc8d281474c9c9240651b93f0feb0fed3d8e949',
  '4f652892b31c421174d7cc66ec88d065bf3b4528fb3bfbce628d2a5edafdcfa',
  '4e8253e27fd60475e67ad462aba22140659171aea360742c551f5769fc72a',
  '8ff4eadc6ab82531196bfc56c527b568b78eb610203cf1117a7dad2ab6932a42',
  'cb9e8cec68248a511ef4e6a7b6a97a3458152e8189e394f2f11388c9ffaf64',
  'e88ed9ed89c2ac3b191dd484634c2276711f116beacedd58bb36572113e15f5f',
  'd88fc8d85a51de83abd6a8420c6ca79982df0f6d268abed743ca5d0d5b23e9a',
  '17c94788a9e95b3e516e1e3b2a29183f84221194408bfac81caabce232674ff',
  'b18cc10f384b5492a63fc0114cc08ff63b1dac9d3b0679a9b7dc54191eaa',
  '4a0773cdbba842d2d5a2aab854af1f53a8bdea45f0c32baadb95652ed72df'])]

,name,url,ram,rom,price,createdDate,color,status,url_img,store,id
0,IPHONE 14 PRO MAX,https://www.xtmobile.vn/iphone-14-pro-max-chinh-hang,6,128,31790000.0,2022-12-26 20:11:52.196381,YELLOW,1,unknown,xtmobile,d88fc8d85a51de83abd6a8420c6ca79982df0f6d268abed743ca5d0d5b23e9a
1,IPHONE 14 PRO MAX,https://www.xtmobile.vn/iphone-14-pro-max-chinh-hang,6,128,32290000.0,2022-12-26 20:11:52.196381,BLACK,1,unknown,xtmobile,5b41f665b4d2c587ce10fedc8d281474c9c9240651b93f0feb0fed3d8e949
2,IPHONE 14 PRO MAX,https://www.xtmobile.vn/iphone-14-pro-max-chinh-hang,6,128,33190000.0,2022-12-26 20:11:52.196381,SILVER,1,unknown,xtmobile,4fb12c7d514ce576c38f8a848819ae50fd22f0c53dda88571fc1239c548a
3,IPHONE 14 PRO MAX,https://www.xtmobile.vn/iphone-14-pro-max-chinh-hang,6,128,33490000.0,2022-12-26 20:11:52.196381,PURPLE,1,unknown,xtmobile,4fd21b74f022929833ac1a2ab2633ce43d4b2dad0a7bd49138bf4c498fc5c24
211,IPHONE 14 PRO MAX,https://www.xtmobile.vn/iphone-14-pro-max-128gb-likenew,6,128,28490000.0,2022-12-26 20:11:52.196381,BLACK,1,unknown,xtmobile,b18cc10f384b5492a63fc0114cc08ff63b1dac9d3b0679a9b7dc54191eaa
212,IPHONE 14 PRO MAX,https://www.xtmobile.vn/iphone-14-pro-max-128gb-likenew,6,128,28790000.0,2022-12-26 20:11:52.196381,YELLOW,1,unknown,xtmobile,17c94788a9e95b3e516e1e3b2a29183f84221194408bfac81caabce232674ff
213,IPHONE 14 PRO MAX,https://www.xtmobile.vn/iphone-14-pro-max-128gb-likenew,6,128,29990000.0,2022-12-26 20:11:52.196381,PURPLE,1,unknown,xtmobile,cb9e8cec68248a511ef4e6a7b6a97a3458152e8189e394f2f11388c9ffaf64
1223,IPHONE 14 PRO MAX,https://24hstore.vn/iphone-14-pro-max-moi/iphone-14-pro-max-1tb-p6626,6,128,31790000.0,2022-12-26 19:39:48.431188,YELLOW,0,unknown,24hstore,e88ed9ed89c2ac3b191dd484634c2276711f116beacedd58bb36572113e15f5f
1295,IPHONE 14 PRO MAX,https://24hstore.vn/iphone-14-pro-max-moi/iphone-14-pro-max-512gb-p6628,6,128,31790000.0,2022-12-26 19:39:48.431188,YELLOW,0,unknown,24hstore,8ff4eadc6ab82531196bfc56c527b568b78eb610203cf1117a7dad2ab6932a42
1355,IPHONE 14 PRO MAX,https://24hstore.vn/iphone-14-pro-max-cu/iphone-14-pro-max-256gb-cu-p7217,6,128,27090000.0,2022-12-26 19:39:48.431188,YELLOW,0,unknown,24hstore,4e8253e27fd60475e67ad462aba22140659171aea360742c551f5769fc72a


In [27]:
cluster[0]

['IPHONE 14 PRO MAX',
 'https://www.xtmobile.vn/iphone-14-pro-max-chinh-hang',
 6,
 128,
 31790000.0,
 '2022-12-26 20:11:52.196381',
 'YELLOW',
 1,
 'unknown',
 'xtmobile',
 'd88fc8d85a51de83abd6a8420c6ca79982df0f6d268abed743ca5d0d5b23e9a',
 ['4f652892b31c421174d7cc66ec88d065bf3b4528fb3bfbce628d2a5edafdcfa',
  'd88fc8d85a51de83abd6a8420c6ca79982df0f6d268abed743ca5d0d5b23e9a',
  '17c94788a9e95b3e516e1e3b2a29183f84221194408bfac81caabce232674ff',
  '4a0773cdbba842d2d5a2aab854af1f53a8bdea45f0c32baadb95652ed72df',
  '8ff4eadc6ab82531196bfc56c527b568b78eb610203cf1117a7dad2ab6932a42',
  '4e8253e27fd60475e67ad462aba22140659171aea360742c551f5769fc72a',
  'e88ed9ed89c2ac3b191dd484634c2276711f116beacedd58bb36572113e15f5f']]

In [28]:
newDf = pd.DataFrame(cluster, columns=['name', 'url', 'ram', 'rom', 'price', 'createdDate', 'color', 'status', 'url_img','store', 'id_orgin', 'list_id'])

In [29]:
newDf.head()

,name,url,ram,rom,price,createdDate,color,status,url_img,store,id_orgin,list_id
0,IPHONE 14 PRO MAX,https://www.xtmobile.vn/iphone-14-pro-max-chinh-hang,6,128,31790000.0,2022-12-26 20:11:52.196381,YELLOW,1,unknown,xtmobile,d88fc8d85a51de83abd6a8420c6ca79982df0f6d268abed743ca5d0d5b23e9a,"[4f652892b31c421174d7cc66ec88d065bf3b4528fb3bfbce628d2a5edafdcfa, d88fc8d85a51de83abd6a8420c6ca79982df0f6d268abed743ca5d0d5b23e9a, 17c94788a9e95b3e516e1e3b2a29183f84221194408bfac81caabce232674ff, 4a0773cdbba842d2d5a2aab854af1f53a8bdea45f0c32baadb95652ed72df, 8ff4eadc6ab82531196bfc56c527b568b78eb610203cf1117a7dad2ab6932a42, 4e8253e27fd60475e67ad462aba22140659171aea360742c551f5769fc72a, e88ed9ed89c2ac3b191dd484634c2276711f116beacedd58bb36572113e15f5f]"
1,IPHONE 14 PRO MAX,https://www.xtmobile.vn/iphone-14-pro-max-chinh-hang,6,128,32290000.0,2022-12-26 20:11:52.196381,BLACK,1,unknown,xtmobile,5b41f665b4d2c587ce10fedc8d281474c9c9240651b93f0feb0fed3d8e949,"[eae6a4e3436cabf1757bd5e619c4436e907fcd5fce681b3b55c232e8ea, 2048717dcc3b475110f3bdd4ac4c1a96d0a923b75ee8d1ee3ff896b248177f2, d497ec96825865c32bdd71fe6ea09ea9764fba45472c9898f1186f8d5989359, 1d64cd68634a11c876a35b7642821a3fb055fa6dd213fdddde5987a6333753c, 423ab36047f7f342fb44ecbf3ccc21fea671c6a47ee3d1be57d0727937ed85bb, b18cc10f384b5492a63fc0114cc08ff63b1dac9d3b0679a9b7dc54191eaa, 5b41f665b4d2c587ce10fedc8d281474c9c9240651b93f0feb0fed3d8e949, 43a4d1507b8030ff143d82b920e587a2aa16d08f419b7965fb9f6d6e6b63ea31]"
2,IPHONE 14 PRO MAX,https://www.xtmobile.vn/iphone-14-pro-max-chinh-hang,6,128,33190000.0,2022-12-26 20:11:52.196381,SILVER,1,unknown,xtmobile,4fb12c7d514ce576c38f8a848819ae50fd22f0c53dda88571fc1239c548a,"[4fb12c7d514ce576c38f8a848819ae50fd22f0c53dda88571fc1239c548a, 4e8656f31eee9cac39f69526659926534a23c2c627bbb672133d22c9541dbb4, dd3c14b2d1d6e12348b872ef9a1e5135880e846cc1c0935d3cbdd6a01ba58e, ee35269a1ad7798084dad77be057927b43f73d2124aa2c4b1a88ade1c7df, 48cbbdcd3f1ecfde4b9ce79139f12bc8d4871342bbe7d9c1eec1fcc8cbdb91e8, 38b537c3ad56376fbb3bf1426d344161ec8c6b8fa69068f9219a08e656045cb, d04a2f373b4914d746f3d311fba5a4d659da1bb1d9f2146769234ed607b26, 156eae42114b79ad2b7641c45f76047efc9cc5ae3d6c65fae9072d05a1777, 79c48dbed0821015365f421d43472c517f5f0309717e0d1e05b2ce5ab50c13c, 8294cc130f56b866a5fe465f8c183ffbeb6158d7d27f4d05d3be0cc4fa12a]"
3,IPHONE 14 PRO MAX,https://www.xtmobile.vn/iphone-14-pro-max-chinh-hang,6,128,33490000.0,2022-12-26 20:11:52.196381,PURPLE,1,unknown,xtmobile,4fd21b74f022929833ac1a2ab2633ce43d4b2dad0a7bd49138bf4c498fc5c24,"[cb9e8cec68248a511ef4e6a7b6a97a3458152e8189e394f2f11388c9ffaf64, 289bd7f8e7982d3bbc988f86cfcc2376e9f9d76a840d238c26fb42e94559e9, 69142a9a87cb5a656339eb14fe51ed93fe55dd798bf2bfb2edd8aa91b8fbb99, 56a0830bd79721cb36160276b71d8f9116e67db9f7b97669995911d88d71, 826cf7737d25a026a58f1366d6723c614b40ae6ae63010af73d586cd6c9d, 606b4fc987827a211d77592d08b7db5bc8b5923c1bd19195db8bed36d722f, 4fd21b74f022929833ac1a2ab2633ce43d4b2dad0a7bd49138bf4c498fc5c24]"
4,IPHONE 14 PRO MAX,https://www.xtmobile.vn/iphone-14-pro-max-256gb,6,256,34590000.0,2022-12-26 20:11:52.196381,YELLOW,1,unknown,xtmobile,dd92229feca528d039b2be9cc962f0c877fe697594881c5ed4b499e7dcfcfef1,"[de966111e543f50fe6a446acd345270341226280b333b61b277690d07721a0, dd92229feca528d039b2be9cc962f0c877fe697594881c5ed4b499e7dcfcfef1, ae1a13868b4db3fbc64113cb70dd1153a61dd2cd958e6885e3aee703714746e, 7f3cbf257fb31aa5af0457184d5825fe0fd9ab865bf7a7f4e81cf34cfadff16, 1acf191f53252fadd11c29275ded3a5e77d8c37fb873612f9d6bc5ea5b15be, b557828e5098296c2e8d67426b2df18116c54b734083583ea5c153115894, 3640290cfb5d4a7a9187180f5c81b70d81de5126ebc60d3a655dfca9a714b72, 8fbe67764b937c70c1e93739483c1db865bcf494ef73e6584e63d740d7e0, 264052899bf516aab2f13f69f6d2975831c428ebfc720ec47f718d8ebfa42dd]"


In [31]:
newDf.to_json('./db/search.json', orient='records')

In [ ]:
df.to_json('./db/products.json', orient='records')